<a href="https://colab.research.google.com/github/Hyeon-Kang/NLP/blob/master/week05_1%20Stemming_Text_classification(2)/(%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D_%EC%98%88%EC%8B%9C)Text_Classfication_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Text classification Example

In [0]:
# Data Load

import csv

file = open ("./IMDB_Dataset.csv", "r", encoding='UTF-8')  # ./ 기본경로는 Script가 저장되는 장소
rdr = csv.reader(file)

sentence = []
label = []

for index, line in enumerate(rdr):
    if index == 0:  # 첫 줄은 필요 없으므로 생략
        continue
    sentence.append(line[0].strip()) # sentence 리스트에 리뷰 내용 추가, strip() 함수는 문자열 양쪽의 공백 제거 (x)
    label.append(line[1].strip())    # label 리스트에 긍정, 부정 추가 (y)

In [0]:
# print data, not preprocessing
print(sentence[0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [0]:
# re 라이브러리로 특수문자 제거

import re

for i in range(len(sentence)):
    sentence[i] = sentence[i].lower() # lower() : 모든 문자를 소문자로 변환
    sentence[i] = re.sub("(<br\s*/><br\s*/>)|(\-)|(\/)", ' ', sentence[i]) # 특수문자 제거
    sentence[i] = re.sub("[.;:!\'?,\"()|[\]]", ' ', sentence[i]) # 특수문자 제거

In [0]:
# re 라이브러리로 불필요 문자 전처리 완료
print(sentence[0])

one of the other reviewers has mentioned that after watching just 1 oz episode you ll be hooked  they are right  as this is exactly what happened with me  the first thing that struck me about oz was its brutality and unflinching scenes of violence  which set in right from the word go  trust me  this is not a show for the faint hearted or timid  this show pulls no punches with regards to drugs  sex or violence  its is hardcore  in the classic use of the word  it is called oz as that is the nickname given to the oswald maximum security state penitentary  it focuses mainly on emerald city  an experimental section of the prison where all the cells have glass fronts and face inwards  so privacy is not high on the agenda  em city is home to many  aryans  muslims  gangstas  latinos  christians  italians  irish and more    so scuffles  death stares  dodgy dealings and shady agreements are never far away  i would say the main appeal of the show is due to the fact that it goes where other shows 

In [0]:
# split data

# 학습 데이터에 40000 할당 (총 50000)
reviews_train_clean = sentence[:40000]    # 0 ~ 40000
train_y = label[:40000]                   

# 나머지 10000은 검증 데이터 할당
reviews_test_clean = sentence[40000:]     # 40000 ~ 50000
test_y = label[40000:]



In [0]:
# CountVectorizer : 문서 집합에서 단어 토큰을 생성, 각 단어의 수를 세어 bag of words로 인코딩한 벡터 생성(단어 빈도 수 측정)

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True) # 성능 개선을 위한 binary 이진분류 옵션 추가, 각 단어의 수를 세어 bag of words 벡터로 인코딩
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

# Train 데이터와 test 데이터를 모두 One-hot 인코딩

In [0]:
# LogistiRegression 회귀분석을 이용한 최적의 파라미터 찾기
# 기본적으로 이항분포 규칙을 따름. 모델의 예측값이 0.5 미만이면 0, 0.5 이상이면 1
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# X_train과 X_validation을 0.75 : 0.25로 나뉘어준다.
X_train, X_val, y_train, y_val = train_test_split(X, train_y, test_size = 0.25, random_state = 17)

for c in[0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C = c)  # 학습에 필요한 최적값 c 찾기
    lr.fit(X_train, y_train)
    print("C = ", c,", Accuracy :", accuracy_score(y_val, lr.predict(X_val))) # C 값의 변화에 따른 정확도 출력

C =  0.01 , Accuracy : 0.8809
C =  0.05 , Accuracy : 0.8872
C =  0.25 , Accuracy : 0.8861
C =  0.5 , Accuracy : 0.8845
C =  1 , Accuracy : 0.8823


In [0]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X_train, y_train) # 찾은 최적의 파라미터로 다시 학습

print("Final Accuracy : ", accuracy_score(test_y, final_model.predict(X_test)))

Final Accuracy :  0.8884


In [0]:
reviews_new = ['This movie was excellent', 'absolute joy ride',
              'Steven Seagal was terrible', 'Steven Seagal shined through.',
              'this was certainly a movie', 'Two thumbs up', 'I fell asleep halfway through',
              "We can't wait for the sequel!!", 'I can recommend this highly enough',
              'instant classic.', 'Steven Seagal was amazing. His perfomance was Oscar-worthy.',
              'I love it']

reviews_new_counts = cv.transform(reviews_new) # CountVectorize로 One-hot 인코딩

In [0]:
pred = final_model.predict(reviews_new_counts)

for review, category in zip(reviews_new,pred):
    print(review," -> ", category)

This movie was excellent  ->  positive
absolute joy ride  ->  positive
Steven Seagal was terrible  ->  negative
Steven Seagal shined through.  ->  negative
this was certainly a movie  ->  negative
Two thumbs up  ->  positive
I fell asleep halfway through  ->  negative
We can't wait for the sequel!!  ->  negative
I can recommend this highly enough  ->  positive
instant classic.  ->  positive
Steven Seagal was amazing. His perfomance was Oscar-worthy.  ->  positive
I love it  ->  positive
